<a href="https://colab.research.google.com/github/siondaniel/Benetech/blob/main/Benetech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy

# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
!git clone https://github.com/siondaniel/Benetech.git

fatal: destination path 'Benetech' already exists and is not an empty directory.


In [30]:
import os

# Define paths to your image and json files
images_dir = "Benetech/train/images"
json_dir = "Benetech/train/annotations"

# Define the list that will hold your data
data = []

# Loop through all the files in the images directory
i = 0
for filename in os.listdir(images_dir):
    if i == 1000: break
    if filename.endswith(".jpg"):
        # Load the image using PIL
        img = Image.open(os.path.join(images_dir, filename))

        # Resize the image to a common size and convert to NumPy array
        img = numpy.array(transforms.functional.resize(img, (256, 256)))
        
        # Load the corresponding json file
        json_filename = filename.split(".")[0] + ".json"
        with open(os.path.join(json_dir, json_filename), "r") as f:
            json_data = json.load(f)
        
        # Append the image and json data to your list
        data.append({
            "image": img,
            "json": json_data
        })
    i+=1

# Print the length of your data
print("Number of data samples:", len(data))

Number of data samples: 1000


In [32]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_data = data[:800]
val_data = data[800:]

# Convert the data into PyTorch tensors
train_images = torch.stack([torch.Tensor(d["image"]) for d in train_data])
train_jsons = [d["json"] for d in train_data]
val_images = torch.stack([torch.Tensor(d["image"]) for d in val_data])
val_jsons = [d["json"] for d in val_data]

# Print the length of your data
print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))

Number of training samples: 800
Number of validation samples: 200


In [33]:
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self, images, targets):
        self.images = images
        self.targets = targets
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, index):
        return self.images[index], self.targets[index]

train_data = MyDataset(train_images, train_jsons)
val_data = MyDataset(val_images, val_jsons)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [34]:
class CNN(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size=5)
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5)
        self.fc1 = nn.Linear(n_feature*4*4, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [35]:
model = CNN(input_size=(3, 224, 224), n_feature=32, output_size=1)

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [38]:
# NEED TO MAKE JSONS EQUAL SIZE AT BEGINNING

num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print statistics
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

RuntimeError: ignored